# Introduction

This notebook shows an example of relaxing the LiFePO4 crystal.

If you are running this notebook from Google Colab, uncomment the next code box to install m3gnet first.

> Note: Because Google Colab's python version is sometimes rather old (v3.7 at the time of writing this notebook (Jul 2022), you will need to install m3gnet without dependencies and install the dependencies manually.

In [2]:
# !pip install --no-deps m3gnet
# !pip install pymatgen ase
# !pip install lxml

In [3]:
import warnings

import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.util.coord import pbc_diff

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")

/Users/shyue/repos/pymatgen/pymatgen/analysis/phase_diagram.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
import tensorflow as tf

tf.config.experimental.list_physical_devices("GPU")

[]

In [5]:
mpr = MPRester()
lfp = mpr.get_structure_by_material_id("mp-19017")  # This is LiFePO4.

lfp_strained = lfp.copy()  # We create a copy.
# Create a random strain between -5% and 5% for each direction
strains = np.random.uniform(low=-0.05, high=0.05, size=3)
lfp_strained.apply_strain(strains)
# In addition to the lattice strains, we also perturb the atoms by a distance of 0.1 angstrom.
lfp_strained.perturb(0.1)

In [6]:
relaxer = Relaxer()
relax_results: dict
%time relax_results = relaxer.relax(lfp_strained)
relaxed = relax_results["final_structure"]

2022-07-22 20:45:58.196042: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


CPU times: user 22.6 s, sys: 45.3 s, total: 1min 7s
Wall time: 12.4 s


Note that the relaxation only took < 20s.

In [7]:
print(f"Original lattice parameters are {[round(x, 3) for x in lfp.lattice.abc]}")
print(f"Strained lattice parameters are {[round(x, 3) for x in lfp_strained.lattice.abc]}")
print(f"Relaxed lattice parameters are {[round(x, 3) for x in relaxed.lattice.abc]}")

Original lattice parameters are [4.746, 10.444, 6.09]
Strained lattice parameters are [4.703, 10.778, 6.1]
Relaxed lattice parameters are [4.751, 10.442, 6.101]


In [8]:
print(f"Diff in fractional coords:\n{pbc_diff(lfp.frac_coords, relaxed.frac_coords)}")

Diff in fractional coords:
[[-1.24366656e-02  1.85275129e-03  5.20024606e-03]
 [-1.25118393e-02 -3.72522910e-03 -2.13609677e-03]
 [-1.09451357e-02 -3.07392613e-03  5.74689067e-03]
 [-8.29439397e-03  4.24244464e-03 -1.29738354e-03]
 [ 3.59255647e-03 -6.87444043e-04 -8.19711225e-03]
 [ 4.03143376e-03  1.30471769e-03  5.32806936e-03]
 [-1.00678750e-02 -1.83023072e-05 -1.22652012e-03]
 [-4.28236250e-03  6.54308196e-04  4.88745868e-03]
 [-5.71330233e-03  1.58379616e-03 -7.89711428e-03]
 [ 1.79388063e-03 -3.40293111e-04  4.48431738e-03]
 [-6.55289857e-03  1.38864162e-03  1.49183186e-03]
 [ 8.71742468e-04 -3.18599684e-04  3.84851613e-03]
 [-4.27816399e-03  1.69104008e-04 -7.54295016e-03]
 [ 2.99860222e-03  1.57156588e-03  4.81418799e-03]
 [-7.03248109e-03  2.19691562e-03 -1.45966073e-03]
 [ 5.04558442e-04  1.08318900e-05  6.93702016e-03]
 [-1.04580632e-02  8.72155903e-04  4.25775575e-03]
 [ 1.53669916e-03  1.75831304e-04 -7.53960221e-03]
 [-6.47354864e-03  1.71527051e-03 -3.49466167e-05]
 [ 6

Quite clealy, the relaxation using the M3GNet universal IAP has brought the lattice parameters much closer to the original DFT one and the coordinates are also within $10^{-3}$ of the original fractional coordinates.